In [2]:
# !pip install pandas

In [3]:
import sqlite3
import random
#import pandas as pd

In command line: 

```
#launch sqlite command line tool & create database called ObligationsDB
sqlite3 ObligationsDB  
#launch sqlite command line tool & create database called InsightsDB
sqlite3 InsightsDB  

# tab-delimted file
.separator ";"

# create new table called "dataObligations" w/ the content from our csv
.import obligations.csv dataObligations
# create new table called "dataInsights" w/ the content from our csv
.import insights.csv dataInsights
```

In [4]:
# Next up: connecting to our data
# online tutorial: https://www.sqlitetutorial.net/sqlite-python/sqlite-python-select/

In [5]:
# add .csv to sqlite table
import pandas as pd 
import sqlite3

# Read values from obligations csv
conn_obligations = sqlite3.connect("./obligationsDB")
dtypes = {'DATEKEY': int, 'ID': str, 'TYPE': str, 'VALUE_TO_PAY': float, 'PAYMENT_DATE': str}
parseDates = ['PAYMENT_DATE']
data_obligations = pd.read_csv('../obligations.csv', sep=';', dtype=dtypes, parse_dates=parseDates)
data_obligations.to_sql("dataObligations", conn_obligations, if_exists='append', index=False)

# Read values from insights csv
conn_insights = sqlite3.connect("./insightsDB")
dtypes = {'ID': str, 'TYPE': str, 'MESSAGE': str}
data_insights = pd.read_csv('../insights.csv', sep=';', dtype=dtypes)
data_insights.to_sql("dataInsights", conn_insights, if_exists='append', index=False)


In [6]:
# delete everything from data_obligations
import sqlite3

#sql = 'DELETE FROM dataObligations'
#conn_obligations = sqlite3.connect("obligationsDB")
#cur_obligations = conn_obligations.cursor()
#cur_obligations.execute(sql)
#conn_obligations.commit()

# delete everything from data_insights
#sql = 'DELETE FROM dataInsights'
#conn_insights = sqlite3.connect("insightssDB")
#cur_insights = conn_insights.cursor()
#cur_insights.execute(sql)
#conn_insights.commit()

In [7]:
def create_connection_obligations(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn_obligations = None
    try:
        conn_obligations = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn_obligations

def create_connection_insights(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn_insights = None
    try:
        conn_insights = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn_insights
'''
def get_everything(conn):
    cursor = conn.execute('select * from dataObligations')
    names = list(map(lambda x: x[0], cursor.description))

    print(names)
'''

def select_by_slot_obligations(conn_obligations, slot_name, slot_value):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur_obligations = conn_obligations.cursor()
    cur_obligations.execute(f"""SELECT * FROM dataObligations
                WHERE {slot_name}='{slot_value}'""")

    rows = cur_obligations.fetchall()

    if len(list(rows)) < 1:
        print("There are no obligations matching your query.")
    else:
        for row in random.sample(rows, 1):
            print(f"Try the {(row[4]).lower()} {row[0]} by {row[1]}. You can find it at {row[2]}.")

def select_by_slot_insights(conn_insights, slot_name, slot_value):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur_insights = conn_insights.cursor()
    cur_insights.execute(f"""SELECT * FROM dataInsights
                WHERE {slot_name}='{slot_value}'""")

    rows = cur_insights.fetchall()

    if len(list(rows)) < 1:
        print("There are no obligations matching your query.")
    else:
        for row in random.sample(rows, 1):
            print(f"Try the {(row[3]).lower()} {row[0]} by {row[1]}. You can find it at {row[2]}.")

#get_everything(create_connection("./obligationsDB"))
select_by_slot_obligations(create_connection_obligations("./obligationsDB"),
    slot_name="ID",slot_value="Deductions")

select_by_slot_insights(create_connection_insights("./insightsDB"),
    slot_name="ID",slot_value="INSIGHT1")

There are no obligations matching your query.
Try the  in the next 30 days, it will increase the disponibilities in 6000€ 202104 by INSIGHT1. You can find it at POSITIVE.


In [12]:
# This files contains your custom actions which can be used to run
# custom Python code.
#
# See this guide on how to implement these action:
# https://rasa.com/docs/rasa/custom-actions

from typing import Any, Text, Dict, List

from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher

import sqlite3
import random


class QueryObligationType(Action):

    def name(self) -> Text:
        return "query_obligation_type"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        conn_obligations = create_connection_obligations(self, db_file="../primavera_db/obligationsDB")

        slot_value = tracker.get_slot("obligation_type")
        slot_name = "ID"

        get_query_results = select_by_slot_obligations(self, conn=conn_obligations,
            slot_name=slot_name,slot_value=slot_value)
        dispatcher.utter_message(text=get_query_results)

        return []

    def create_connection_obligations(db_file):
        """ create a database connection to the SQLite database
            specified by the db_file
        :param db_file: database file
        :return: Connection object or None
        """
        conn_obligations = None
        try:
            conn_obligations = sqlite3.connect(db_file)
        except Error as e:
            print(e)

        return conn_obligations

    def select_by_slot_obligations(conn_obligations, slot_name, slot_value):
        """
        Query all rows in the tasks table
        :param conn: the Connection object
        :return:
        """
        cur_obligations = conn_obligations.cursor()
        cur_obligations.execute(f"""SELECT * FROM dataObligations
                    WHERE {slot_name}='{slot_value}'""")

        rows = cur_obligations.fetchall()

        if len(list(rows)) < 1:
            return[("There are no obligations matching your query.")]
        else:
            for row in random.sample(rows, 1):
                return[print(f"Try the {(row[4]).lower()} {row[0]} by {row[1]}. You can find it at {row[2]}.")]

class QueryInsightType(Action):

    def name(self) -> Text:
        return "query_insight_type"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        conn_insights = create_connection_insights(self, db_file="../primavera_db/insightsDB")

        slot_value = tracker.get_slot("insight_type")
        slot_name = "ID"

        get_query_results = select_by_slot_obligations(self, conn=conn_insights,
            slot_name=slot_name,slot_value=slot_value)
        dispatcher.utter_message(text=get_query_results)

        return []

    def create_connection_insights(db_file):
        """ create a database connection to the SQLite database
            specified by the db_file
        :param db_file: database file
        :return: Connection object or None
        """
        conn_insights = None
        try:
            conn_insights = sqlite3.connect(db_file)
        except Error as e:
            print(e)

        return conn_insights

    def select_by_slot_insights(conn_insights, slot_name, slot_value):
        """
        Query all rows in the tasks table
        :param conn: the Connection object
        :return:
        """
        cur_insights = conn_insights.cursor()
        cur_insights.execute(f"""SELECT * FROM dataInsights
                    WHERE {slot_name}='{slot_value}'""")
        rows = cur_insights.fetchall()

        if len(list(rows)) < 1:
            return[("There are no insights matching your query.")]
        else:
            for row in random.sample(rows, 1):
                return[print(f"Try the {(row[3]).lower()} {row[0]} by {row[1]}. You can find it at {row[2]}.")]

In [15]:
# from: https://techoverflow.net/2019/10/14/how-to-list-tables-in-sqlite3-database-in-python/
def tables_in_sqlite_db_obligations(conn_obligations):
    cursor_obligations = conn_obligations.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables_obligations = [
        v[0] for v in cursor_obligations.fetchall()
        if v[0] != "sqlite_sequence"
    ]
    cursor_obligations.close()
    return tables_obligations

def tables_in_sqlite_db_insights(conn_insights):
    cursor_insights = conn_insights.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables_insights = [
        v[0] for v in cursor_insights.fetchall()
        if v[0] != "sqlite_sequence"
    ]
    cursor_insights.close()
    return tables_insights

In [18]:
conn_obligations = QueryObligationType.create_connection_obligations("../obligationsDB")
print(tables_in_sqlite_db_obligations(conn_obligations))
conn_insights = QueryInsightType.create_connection_insights("../insightsDB")
print(tables_in_sqlite_db_insights(conn_insights))

[]
[]
